In [6]:
# Step 1: Import libraries
import pandas as pd
import os

# Step 2: Set file paths
RAW_DIR = "/Users/sachinyaduwanshi/Desktop/sales_forecasting/data/raw/"

# Step 3: Load CSVs
train = pd.read_csv(os.path.join(RAW_DIR, "train.csv"))
features = pd.read_csv(os.path.join(RAW_DIR, "features.csv"))
stores = pd.read_csv(os.path.join(RAW_DIR, "stores.csv"))
test = pd.read_csv(os.path.join(RAW_DIR, "test.csv"))

# Step 4: Convert Date columns to datetime
train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])
test['Date'] = pd.to_datetime(test['Date'])

# Step 5: Quick inspection
print("Train shape:", train.shape)
print("Features shape:", features.shape)
print("Stores shape:", stores.shape)
print("Test shape:", test.shape)




# Step 6: Preview first 5 rows
print("\n--- Train ---")
display(train.head())

print("\n--- Features ---")
display(features.head())

print("\n--- Stores ---")
display(stores.head())

print("\n--- test---")
display(test.head())

# Step 7: Check for missing values
print("\n--- Missing values in Train ---")
print(train.isnull().sum())

print("\n--- Missing values in Features ---")
print(features.isnull().sum())

print("\n--- Missing values in Stores ---")
print(stores.isnull().sum())

print("\n--- Missing values in Test ---")
print(test.isnull().sum())


Train shape: (421570, 5)
Features shape: (8190, 12)
Stores shape: (45, 3)
Test shape: (115064, 4)

--- Train ---


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False



--- Features ---


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False



--- Stores ---


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875



--- test---


,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False



--- Missing values in Train ---
Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

--- Missing values in Features ---
Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

--- Missing values in Stores ---
Store    0
Type     0
Size     0
dtype: int64

--- Missing values in Test ---
Store        0
Dept         0
Date         0
IsHoliday    0
dtype: int64


In [8]:
# ==============================
# STEP 8: DATASET MERGING
# ==============================

# Merge train with features
df = train.merge(
    features,
    on=["Store", "Date", "IsHoliday"],
    how="left"
)

# Merge store information
df = df.merge(
    stores,
    on="Store",
    how="left"
)

print("Final merged dataset shape:", df.shape)


Final merged dataset shape: (421570, 16)


In [10]:
# ==============================
# MISSING VALUE HANDLING
# ==============================

# MarkDown columns
# NaN means no promotion was running
markdown_cols = [
    "MarkDown1", "MarkDown2", "MarkDown3",
    "MarkDown4", "MarkDown5"
]

df[markdown_cols] = df[markdown_cols].fillna(0)

# CPI and Unemployment
# Forward fill because they change slowly over time
df["CPI"] = df["CPI"].fillna(method="ffill")
df["Unemployment"] = df["Unemployment"].fillna(method="ffill")

# Safety check (if first rows still NaN)
df["CPI"] = df["CPI"].fillna(method="bfill")
df["Unemployment"] = df["Unemployment"].fillna(method="bfill")

# Final check
print(df.isnull().sum())


Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type            0
Size            0
dtype: int64


/var/folders/ff/5tkb80t94wx7qkxv6ytqyvsw0000gn/T/ipykernel_4896/381964024.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["CPI"] = df["CPI"].fillna(method="ffill")
/var/folders/ff/5tkb80t94wx7qkxv6ytqyvsw0000gn/T/ipykernel_4896/381964024.py:17: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Unemployment"] = df["Unemployment"].fillna(method="ffill")
/var/folders/ff/5tkb80t94wx7qkxv6ytqyvsw0000gn/T/ipykernel_4896/381964024.py:20: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["CPI"] = df["CPI"].fillna(method="bfill")
/var/folders/ff/5tkb80t94wx7qkxv6ytqyvsw0000gn/T/ipykernel_4896/381964024.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bf

In [11]:
# ==============================
# SAVE CLEANED DATA
# ==============================

PROCESSED_DIR = "/Users/sachinyaduwanshi/Desktop/sales_forecasting/data/processed/"

os.makedirs(PROCESSED_DIR, exist_ok=True)

df.to_csv(
    os.path.join(PROCESSED_DIR, "sales_cleaned.csv"),
    index=False
)

print("✅ Processed dataset saved successfully")
print("Shape:", df.shape)


✅ Processed dataset saved successfully
Shape: (421570, 16)
